In [4]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate
from transformers import pipeline

ModuleNotFoundError: No module named 'datasets'

In [2]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv('../IMDB_dataset/IMDB dataset.csv')

df.sentiment.replace("positive" , 1 , inplace = True)
df.sentiment.replace("negative" , 0 , inplace = True)
df = df.rename(columns={"review": "text", "sentiment":"label"}) 

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
dataset_train = Dataset.from_pandas(train, preserve_index=False)
dataset_test = Dataset.from_pandas(test, preserve_index=False)

In [5]:
dataset_train

Dataset({
    features: ['text', 'label'],
    num_rows: 40000
})

In [6]:
dataset_train[1]

{'text': "I did not watch the entire movie. I could not watch the entire movie. I stopped the DVD after watching for half an hour and I suggest anyone thinking of watching themselves it stop themselves before taking the disc out of the case.<br /><br />I like Mafia movies both tragic and comic but Corky Romano can only be described as a tragic attempt at a mafia comedy.<br /><br />The problem is Corky Romano simply tries too hard to get the audience to laugh, the plot seems to be an excuse for moving Chris Kattan (Corky) from one scene to another. Corky himself is completely overplayed and lacks subtlety or credulity - all his strange mannerisms come across as contrived - Chris Kattan is clearly 'acting' rather than taking a role - it bounces you right out of the story. Each scene is utterly predictable, the 'comedic event' that will occur on the set is obvious as soon as each scene is introduced. In comedies such as Mr. Bean the disasters caused by the title character are funny becaus

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def tokenize(sample):
    return tokenizer(sample["text"], padding="max_length", truncation=True)

In [9]:
dataset_train_tokenized = dataset_train.map(tokenize, batched=True)
dataset_test_tokenized = dataset_test.map(tokenize, batched=True)

In [10]:
dataset_train_tokenized

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 40000
})

In [11]:
dataset_test_tokenized

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [12]:
small_train_dataset = dataset_train_tokenized.shuffle(seed=42).select(range(25000))

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

print(model)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [14]:
metric = evaluate.load("accuracy")

axis = -1

This means that the index that will be returned by argmax will be taken from the last axis.

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
training_args = TrainingArguments(output_dir = "bert", 
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch",
                                  per_device_train_batch_size = 4,
                                  per_device_eval_batch_size = 4,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5)

In [17]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = dataset_test_tokenized,
    compute_metrics = compute_metrics,
)

In [18]:
trainer.train()

C:\Users\wojte\anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.329400,0.363978,0.916700
2,0.168400,0.351262,0.930000


TrainOutput(global_step=12500, training_loss=0.2772796936035156, metrics={'train_runtime': 7582.8893, 'train_samples_per_second': 6.594, 'train_steps_per_second': 1.648, 'total_flos': 6623369932800000.0, 'train_loss': 0.2772796936035156, 'epoch': 2.0})

In [19]:
trainer.save_model("./bert/")

In [20]:
tokenizer.save_pretrained('./bert/tokenizer/')

('./bert/tokenizer/tokenizer_config.json',
 './bert/tokenizer/special_tokens_map.json',
 './bert/tokenizer/vocab.txt',
 './bert/tokenizer/added_tokens.json',
 './bert/tokenizer/tokenizer.json')

### Load model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

C:\Users\wojte\anaconda3\envs\reviews-analysis-api\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_bert_folder = './bert/'
bert_tokenizer_folder = './bert/tokenizer/'

model_bert = AutoModelForSequenceClassification.from_pretrained(model_bert_folder)
bert_tokenizer = AutoTokenizer.from_pretrained(bert_tokenizer_folder)

bert_pipe = pipeline("text-classification", model=model_bert, tokenizer=bert_tokenizer, top_k=None)

In [3]:
review = """Do not look at this through the prism of "Foreign Films". You'd be wasting your time and miss something far too important.

Hollywood does scale like nobody else, leaving the competition gasping in its wake. France does intimacy, and brutality. Nothing is sacred. And rather than try to revive the New Wave or emulate Hollywood like most widely seen French films of late, "Intouchables" harnesses its core strengths - ease with intimacy, willingness to ridicule anything and brutal honesty - and delivers one of the funniest, most honest and touching films I have ever seen.

Sy is a failed robber, going through the motions and playing the stereotypical jobless émigré. Cluzet is a romantic and melancholy mind trapped in a useless body. The circumstances that bring them together are too funny to spoil here, but meet they do, and an awkward relationship quickly blossoms as they bring out the best in each other.

The film's simplicity is delightfully misleading: the script is a masterpiece of comedy writing, and however good the rest of the cast is, the central duo is magical. Sy's comic timing will have you in stitches, but it is his honesty and vulnerability that make you fall in love with the character. Cluzet isn't your typical sad-sack, instead, much of the finest pleasures in the film consist in watching him use his keen mind to mess with the world around him (a subplot about an abstract painting really takes the biscuit, you'll know it when you see it).

This is one of the most unique, beautiful and honest friendships ever committed to film. It will make you laugh, it will make you cry... a delightful celebration of everything in life that makes it worthwhile."""

review2 = """This movie is an absolute slog. It did a terrible job of keeping me interested and invested. It doesn't take much to hook me, but I was barely engaged. I felt this from beginning to end.

So much of this movie feels off. It's filled with eyebrow-raising moments. Like really odd character reaction shots and super awkward editing. Or other weird things, like a character who is listening, so she repeatedly says, "Listening." I couldn't help but laugh.

The only thing keeping me from leaving the theater was finding out the reveal and explanation. Unfortunately it's super lame and kind of dumb. Also, I'm a fan of Tina Fey but she doesn't fit here.

The one positive that stands out to me is it's visually appealing. That's about it. I had a terrible time with this movie. Knives Out is now the clear winner of this mystery movie series rivalry."""
review3 = """This movie is an absolute slog. It did a terrible job of keeping me interested and invested. It doesn't take much to hook me, but I was barely engaged. I felt this from beginning to end.

So much of this movie feels off. It's filled with eyebrow-raising moments. Like really odd character reaction shots and super awkward editing. Or other weird things, like a character who is listening, so she repeatedly says, "Listening." I couldn't help but laugh.

The only thing keeping me from leaving the theater was finding out the reveal and explanation. Unfortunately it's super lame and kind of dumb. Also, I'm a fan of Tina Fey but she doesn't fit here.

The one positive that stands out to me is it's visually appealing. That's about it. I had a terrible time with this movie. Knives Out is now the clear winner of this mystery movie series rivalry."""

In [4]:
predictions = bert_pipe([review, review2, review3])

In [5]:
predictions

[[{'label': 'LABEL_1', 'score': 0.9996761083602905},
  {'label': 'LABEL_0', 'score': 0.000323950604069978}],
 [{'label': 'LABEL_0', 'score': 0.9996267557144165},
  {'label': 'LABEL_1', 'score': 0.00037324457662180066}],
 [{'label': 'LABEL_0', 'score': 0.9996267557144165},
  {'label': 'LABEL_1', 'score': 0.00037324457662180066}]]

In [6]:
output = []
for prediction in predictions:
    pos = 0
    neg = 0
    for score in prediction:
        if score['label'] == 'LABEL_1':
            pos = score['score']
        elif score['label'] == 'LABEL_0':
            neg = score['score']
    output.append({"prob_pos": "{:.2f}".format(float(pos)), "prob_neg": "{:.2f}".format(float(neg))})

print(output)

[{'prob_pos': '1.00', 'prob_neg': '0.00'}, {'prob_pos': '0.00', 'prob_neg': '1.00'}, {'prob_pos': '0.00', 'prob_neg': '1.00'}]


In [5]:
import pandas as pd

In [9]:
df_reviews = pd.read_csv('../IMDB_dataset/IMDB dataset.csv')['review'][0:10]

In [10]:
df_reviews

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
5    Probably my all-time favorite movie, a story o...
6    I sure would like to see a resurrection of a u...
7    This show was an amazing, fresh & innovative i...
8    Encouraged by the positive comments about this...
9    If you like original gut wrenching laughter yo...
Name: review, dtype: object

In [12]:
predictions = bert_pipe([review for review in df_reviews])

In [13]:
predictions

[[{'label': 'LABEL_1', 'score': 0.9988672733306885},
  {'label': 'LABEL_0', 'score': 0.001132776727899909}],
 [{'label': 'LABEL_1', 'score': 0.9996291399002075},
  {'label': 'LABEL_0', 'score': 0.00037088527460582554}],
 [{'label': 'LABEL_1', 'score': 0.9991717338562012},
  {'label': 'LABEL_0', 'score': 0.0008282645721919835}],
 [{'label': 'LABEL_0', 'score': 0.9995326995849609},
  {'label': 'LABEL_1', 'score': 0.0004672856885008514}],
 [{'label': 'LABEL_1', 'score': 0.9996320009231567},
  {'label': 'LABEL_0', 'score': 0.00036793045001104474}],
 [{'label': 'LABEL_1', 'score': 0.9995561242103577},
  {'label': 'LABEL_0', 'score': 0.0004438576870597899}],
 [{'label': 'LABEL_1', 'score': 0.9815404415130615},
  {'label': 'LABEL_0', 'score': 0.018459510058164597}],
 [{'label': 'LABEL_0', 'score': 0.9996428489685059},
  {'label': 'LABEL_1', 'score': 0.0003571833949536085}],
 [{'label': 'LABEL_0', 'score': 0.9997969269752502},
  {'label': 'LABEL_1', 'score': 0.00020313005370553583}],
 [{'label

In [14]:
output = []
for prediction in predictions:
    pos = 0
    neg = 0
    for score in prediction:
        if score['label'] == 'LABEL_1':
            pos = score['score']
        elif score['label'] == 'LABEL_0':
            neg = score['score']
    output.append({"prob_pos": "{:.2f}".format(float(pos)), "prob_neg": "{:.2f}".format(float(neg))})

In [15]:
output

[{'prob_pos': '1.00', 'prob_neg': '0.00'},
 {'prob_pos': '1.00', 'prob_neg': '0.00'},
 {'prob_pos': '1.00', 'prob_neg': '0.00'},
 {'prob_pos': '0.00', 'prob_neg': '1.00'},
 {'prob_pos': '1.00', 'prob_neg': '0.00'},
 {'prob_pos': '1.00', 'prob_neg': '0.00'},
 {'prob_pos': '0.98', 'prob_neg': '0.02'},
 {'prob_pos': '0.00', 'prob_neg': '1.00'},
 {'prob_pos': '0.00', 'prob_neg': '1.00'},
 {'prob_pos': '1.00', 'prob_neg': '0.00'}]

In [21]:
pos = 0
neg = 0

for sentiment in output:
    if float(sentiment['prob_pos']) > 0.5:
        pos += 1
    else:
        neg += 1

In [22]:
pos

7

In [23]:
neg

3